# 序列推荐 (Sequential Recommendation)

## DIN (Deep Interest Network)

- 核心原理：在预测用户是否会点击一个 target item 时，用户的兴趣不应该是一个固定的向量，而应该根据这个目标商品来动态调整。
- DIN 通过一个注意力机制 (Attention Unit) 来实现这种“局部激活” (Local Activation) 的。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Attention Unit
class AttentionUnit(nn.Module):
    def __init__(self, query_dim, key_dim, hidden_units):
        super(AttentionUnit, self).__init__()
        # fc1 的输入维度是 query_dim + key_dim
        # 将目标商品和历史商品拼在一起
        self.fc1 = nn.Linear(query_dim + key_dim, hidden_units)
        # fc2 输出一个单独的“相关性分数”
        self.fc2 = nn.Linear(hidden_units, 1)

    def forward(self, query, keys):
        # query: (batch_size, embedding_size)
        # 目标商品的 embedding

        # keys: (batch_size, seq_len, embedding_size)
        # 历史行为序列的 embedding
        seq_len = keys.size(1)

        # 1. 扩展 query
        # 将目标商品扩展 seq_len 份，使其维度与 key 相同
        # Why？ 为了计算目标商品与每一个历史商品的相关性
        queries = query.unsqueeze(1).expand(-1, seq_len, -1)  # (batch_size, seq_len, embedding_size)

        # 2. 拼接 query 和 keys
        # 这是 DIN 注意力机制的特点：显式地将 query 和 keys 拼接起来
        inputs = torch.cat([queries, keys], dim = -1) # shape: (batch, seq_len, embedding_size * 2)

        # 3. 通过 MLP 计算“相关性”
        # “注意力网络” 是通过 MLP 架构实现的
        out = F.relu(self.fc1(inputs)) # out shape: (batch_size, seq_len, hidden_units)

        # 得到每个历史商品的原始注意力分数
        out = self.fc2(out).squeeze(-1) # shape: (batch_size, seq_len）each batch: (seq_len, ), single original score

        # 4. Softmax 归一化
        attention_weights = F.softmax(out, dim=-1) # shape: (batch_size, seq_len)

        return attention_weights


# DIN implementation

class DIN(nn.Module):
    def __init__(self, user_num, item_num, embedding_dim, attention_hidden_units):
        super(DIN, self).__init__()

        # 1. Embedding 层，存储 user id 和 item id 的 embedding
        self.user_embedding = nn.Embedding(user_num, embedding_dim)
        self.item_embedding = nn.Embedding(item_num, embedding_dim)

        # 2. 实例化 Attention 单元
        self.attention = AttentionUnit(embedding_dim, embedding_dim, attention_hidden_units)

        # 3. 顶层 DNN (Deep Neural Network)
        # 注意：输入维度是 embedding_dim * 3
        self.dnn = nn.Sequential(
            nn.Linear(embedding_dim * 3, 80),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(80, 40),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(40,1)
        )
    
    def forward(self, user, hist_items, target_item):
        # == embedding lookup ==
        # 静态用户特征
        user_emb = self.user_embedding(user) # shape: (batch_size, embedding_dim)

        # 目标商品特征
        target_item_emb = self.item_embedding(target_item) # shape: (batch_szie, embedding_dim)

        # 历史商品序列
        hist_item_emb = self.item_embedding(hist_items) # shape: (batch_size, seq_len, embedding_dim)

        # == Core Steps ==

        # 1. 计算注意力分数
        attention_weights = self.attention(target_item_emb, his_items_emb) # shape: (batch_size, seq_len)

        # 2. 应用注意力权重（加权求和）
        # attention_weights: (batch, seq_len）-> (batch, seq_len, 1)
        # hist_item_emb: (batch, seq_len, embedding_dim)
        # 沿着 seq_len 维度求和，得到一个 (batch, embedding_dim) 的张量，一个 embedding 代表一个历史序列
        weighted_hist_emb = torch.sum(hist_item_emb, attention_weights.unsqueeze(-1), dim = 1)

        # weighted_hist_emb 是 **动态**，**为 target item** 定制的 embedding，融合了当前 item 和用户兴趣

        # == Concat & Prediction

        # 1. 拼接所有特征
        # 拼接 user_emb（静态用户特征）, target_item_emb（目标商品特征） ,weighted_hist_emb（动态用户兴趣）
        dnn_input = torch.cat([user_emb, target_item_emb, weighted_hist_emb], dim=-1) # shape: (batch, embedding_dim * 3)

        # 2. 通过 DNN 预测
        output = self.dnn(dnn_input) # shape: (batch, 1)

        # 3. Sigmoid 输出，作为点击率的预测值
        return torch.sigmoid(output)

